# 正則化 (Regularization)


In [ ]:
# Tensorflowが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env TF_NUM_INTEROP_THREADS=1
%env TF_NUM_INTRAOP_THREADS=1

from tensorflow.config import threading
num_threads = 1
threading.set_inter_op_parallelism_threads(num_threads)
threading.set_intra_op_parallelism_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import utils

## 過学習(Overtraining)の例
### データ点の生成
sin関数に0.3の大きさのガウシアンノイズがのったデータを考えます。データ数は30にします。
早期終了の例と同様に、scikit learningの関数を使ってデータをトレーニング用と検証用に分割します。

In [ ]:
# データ点の取得
x, t = utils.dataset_overtraining(n=30, noise_scale=0.3)

# 学習データと検証用データに分割
from sklearn.model_selection import train_test_split
x_train, x_valid, t_train, t_valid = train_test_split(x, t, test_size=0.30, random_state=0)

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# データ点をプロット
ax.scatter(x_train, t_train, s=10, c="black", label="data")
ax.scatter(x_valid, t_valid, s=10, c="orange", label="data")
x_grid = np.linspace(-np.pi, np.pi, 100)
ax.plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax.legend()

# 図を表示
plt.show()

### デモに用いる深層学習モデル
過学習の様子を見るために、パラメータ数の多いモデルを使ってフィットをしてみます。
ここでは、ノード数が128の隠れ層を5層重ねた多層パーセプトロンを使用します。
活性化関数としてはReLUを使い、モデルの出力の直前のノードは、活性化関数を使用しないことにします。
誤差関数は二乗和誤差を使い、最適化関数としてadamを使用します。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

model = Sequential(
    [
        Input(shape=(1,)),
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(128, activation="relu"),  # ノード数が128の層を追加。活性化関数はReLU関数。
        Dense(1, activation="linear"),  # ノード数が1の層を追加。活性化関数は線形関数。
    ]
)

model.compile(loss="mean_squared_error", optimizer="adam")

#  トレーニング
history = model.fit(x=x_train, y=t_train, validation_data=(x_valid, t_valid), epochs=500, verbose=0)

# ロス関数の推移を取得します
loss_train = history.history["loss"]
loss_valid = history.history["val_loss"]

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train, label="loss (train)")
ax[1].plot(loss_valid, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

## L2 正則化
過学習の問題は、トレーニングサンプルの数が少なすぎる時、もしくはモデルの表現能力が高すぎる、パラメータの自由度が大きすぎると起こります。
校舎を抑制するため、パラメータの自由度を適度に制約してやることで過学習を抑えることができます。

正則化(Regularization)は、パラメータに制限をかけることで、過学習を抑制します。

代表的なものはL2 正則、もしくは *荷重減衰(weight decay)* と呼ばれる手法で、重みパラメータの2乗和
$$
L_{\text L2} = \sum_i w_i^2
$$
の大きさでペナルティーをかけることで、重みパラメータが過度に大きくならないようにします。

(実はこの制約のかけ方は、$w_i$の事前分布としてガウス分布を仮定すると、導出されます。興味のあるかたは調べてみてください。)

Tensorflow/Kerasではこれが簡単に行えるようになっています。


L2正則化のための関数が用意されています。
https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/L2

Dense レイヤーの重みパラメータにL2 ロスを入れたい場合は、
```python
from tensorflow.keras.regularizers import l2
Dense(
    units=128,
    activation='relu',
    kernel_regularizer=l2(),
    bias_regularizer=l2(),
)
```
もしくは、より簡単に
```python
Dense(
    units=128,
    activation='relu',
    kernel_regularizer='l2',
    bias_regularizer='l2',
)
```
のように引数としてどのような正則化を加えるかを指定します。
文字列として正則化項の指定をした場合は、デフォルトの値が使われます。
一方、`l2()`のようにした場合、l2の引数を指定することで、ペナルティの大きさを調整することもできます。 

例えば、`l2(l2=0.01)`のようにした場合、
$$
L_{\text total} = L + 0.01 \cdot \sum_i w_i^2
$$
のようなロス関数を最小化することになります。

実際に学習をさせてみましょう。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.regularizers import l2

model = Sequential(
    [
        Input(shape=(1,)),
        Dense(128, activation="relu", kernel_regularizer=l2(l2=0.01), bias_regularizer=l2(l2=0.01)),
        Dense(128, activation="relu", kernel_regularizer=l2(l2=0.01), bias_regularizer=l2(l2=0.01)),
        Dense(128, activation="relu", kernel_regularizer=l2(l2=0.01), bias_regularizer=l2(l2=0.01)),
        Dense(128, activation="relu", kernel_regularizer=l2(l2=0.01), bias_regularizer=l2(l2=0.01)),
        Dense(128, activation="relu", kernel_regularizer=l2(l2=0.01), bias_regularizer=l2(l2=0.01)),
        Dense(1, activation="linear", kernel_regularizer=l2(l2=0.01), bias_regularizer=l2(l2=0.01)),
    ]
)

model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

# トレーニング
history = model.fit(x=x_train, y=t_train, validation_data=(x_valid, t_valid), epochs=300, verbose=0)

# L2を除いたロス関数の推移を取得します
loss_train = history.history["mse"]
loss_valid = history.history["val_mse"]

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train, label="loss (train)")
ax[1].plot(loss_valid, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

正則化によって過学習が抑制されてます。また検証用データに誤差関数の値が十分に小さく抑えられているはずです。

## L1正則化
もう一つよく使われる正則化手法として、L1 正則化があります。
こちらは、Lasso とも呼ばれることがあります。

L1 正則化では、
$$
L_{\text L1} = \sum_i |w_i|
$$
の形で誤差関数にペナルティーを与えます。
L1正則化は、L2正則化と比較して、$w_i$が0になりやすいという特徴があります。
このことから、モデルパラメータを疎にする目的でこの手法が使われることもあります。

こちらもTensorflow/Keras で関数が用意されており、l2と同様に使うことができます。
https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/L1


In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.regularizers import l1

model = Sequential(
    [
        Input(shape=(1,)),
        Dense(128, activation="relu", kernel_regularizer=l1(l1=1e-3), bias_regularizer=l1(l1=1e-3)),
        Dense(128, activation="relu", kernel_regularizer=l1(l1=1e-3), bias_regularizer=l1(l1=1e-3)),
        Dense(128, activation="relu", kernel_regularizer=l1(l1=1e-3), bias_regularizer=l1(l1=1e-3)),
        Dense(128, activation="relu", kernel_regularizer=l1(l1=1e-3), bias_regularizer=l1(l1=1e-3)),
        Dense(1, activation="linear", kernel_regularizer=l1(l1=1e-3), bias_regularizer=l1(l1=1e-3)),
    ]
)

model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

#  トレーニング
history = model.fit(x=x_train, y=t_train, validation_data=(x_valid, t_valid), epochs=300, verbose=0)

# L1を除いたロス関数の推移を取得します
loss_train = history.history["mse"]
loss_valid = history.history["val_mse"]

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))

# データ点をプロット
ax[0].scatter(x_train, t_train, s=10, c="black", label="training data")
ax[0].scatter(x_valid, t_valid, s=10, c="orange", label="validation data")
ax[0].plot(x_grid, model(x_grid), c="red", label="prediction")
ax[0].plot(x_grid, np.sin(x_grid), c="blue", label="y=sin(x)")
ax[0].legend()

# ロス関数の推移をプロット
ax[1].plot(loss_train, label="loss (train)")
ax[1].plot(loss_valid, label="loss (valid)")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

# 図を表示
plt.show()

L1L2という関数で、L1正則化とL2正則化の両方をかけることもできます。
$$
L_{\text total} = L + \lambda_{L1} \cdot \sum_i |w_i| + \lambda_{L2} \cdot \sum_i w_i^2
$$

https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/L1L2

```python
L1L2(
    l1=0.001,
    l2=0.01
)
```

L1, L2正則化の大きさを変化させて、モデルの予測がどのように変化するかを調べてみましょう。